In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [64]:
df = pd.read_csv('SZFZ_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])
df['DateConfirm'] = df['DateConfirm'].apply(lambda x: str(x.year) + '-' + str(x.month))

df.to_csv('SZFZ_working_monthly_2.csv',index=None)

In [65]:
df.drop('DateConfirm',axis=1,inplace=True)
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])

df['DateSymptom'] = df['DateSymptom'].dt.to_period('M')

date_start = df.sort_values(by='DateSymptom').iloc[0]['DateSymptom']
date_end = df.sort_values(by='DateSymptom').iloc[-1]['DateSymptom']
date_index = pd.PeriodIndex(start=date_start,end=date_end,freq='m')

date_index

df.loc[df['Class']=='SZFZ','Class'] = 0
df.loc[:,'Class']=df.loc[:,'Class'].astype(float)

In [66]:
TYPES = ['I_new','I','A(not tp)','tp','A','I/(I+A)','I/population']
CLASSES = df['Class'].unique()
CLASSES.sort()
CLASSES = CLASSES[::-1]
CLASSES = np.append(CLASSES,[1,])
CLASSES = CLASSES.astype(float)

POPULATION = [1118/18,] * 19
POPULATION = np.append(POPULATION,[1118,])

type_class = [(t,c)for t in TYPES for c in CLASSES]

type_index = pd.MultiIndex.from_tuples(type_class)

org_df = pd.DataFrame(index=date_index,columns=type_index)

org_df[:] = 0

date_length = len(date_index)
for c in CLASSES[:-1]:
    class_df = df[df['Class']==c]
    for t,n in [('I',1),('A(not tp)',2),('tp',3)]:
        cur_df = class_df[class_df['Type']==n]['DateSymptom'].value_counts().sort_index()
        cur_dates = cur_df.index.values
        cur_vals = cur_df.values
        l = 0
        for i in range(date_length):
            if i>0:
                org_df.loc[date_index[i],(t,c)] = org_df.loc[date_index[i-1],(t,c)] * 5/6
            if l<len(cur_dates) and date_index[i]==cur_dates[l]:
                org_df.loc[date_index[i],(t,c)] += cur_vals[l]
                if n==1:
                    org_df.loc[date_index[i],('I_new',c)] = cur_vals[l]
                l += 1

date_length = len(date_index)
for c in CLASSES[:-1]:
    org_df.loc[:,('A',c)] = org_df.loc[:,('A(not tp)',c)] + org_df.loc[:,('tp',c)]

for t in ['I_new','I','A(not tp)','tp','A']:
    org_df.loc[:,(t,1)] = org_df.loc[:,t].iloc[:,:-1].sum(axis=1)

for c,p in zip(CLASSES,POPULATION):
    org_df.loc[:,('I/(I+A)',c)] = org_df.loc[:,('I',c)] / (org_df.loc[:,('I',c)] + org_df.loc[date_index[i],('A',c)])
    org_df.loc[:,('I/population',c)] = org_df.loc[:,('I',c)] / p

In [67]:
org_df=org_df.swaplevel(axis=1)
v = pd.Categorical(org_df.columns.get_level_values(0), 
                   categories=CLASSES, 
                   ordered=True)
v2 = pd.Categorical(org_df.columns.get_level_values(1), 
                    categories=TYPES,
                    ordered=True)
org_df.columns = pd.MultiIndex.from_arrays([v, v2]) 
org_df = org_df.sort_index(axis=1, level=[0, 1])
org_df.fillna(0,inplace=True)
org_df

52.0                                                51.0       \
        I_new    I A(not tp)   tp    A I/(I+A) I/population I_new    I   
2013-12     0  0.0       0.0  0.0  0.0     0.0       0.0000     0  0.0   
2014-01     0  0.0       0.0  0.0  0.0     0.0       0.0000     0  0.0   
2014-02     0  0.0       0.0  0.0  0.0     0.0       0.0000     0  0.0   
2014-03     0  0.0       0.0  0.0  0.0     0.0       0.0000     0  0.0   
2014-04     0  0.0       0.0  0.0  0.0     0.0       0.0000     0  0.0   
2014-05     1  1.0       0.0  0.0  0.0     1.0       0.0161     1  1.0   

                      ...      0.0                        1.0              \
        A(not tp)     ...         A I/(I+A) I/population I_new          I   
2013-12       0.0     ...       0.0     0.0     0.000000     1   1.000000   
2014-01       0.0     ...       0.0     0.0     0.000000     0   0.833333   
2014-02       0.0     ...       0.0     1.0     0.048301     3   3.694444   
2014-03       0.0     ...       0.0     1.0     0.072451     3   6.078704   
2014-04       0.0     ...       0.0     1.0     0.173077    23  28.065586   
2014-05       0.0     ...       0.0     1.0     0.144231    36  59.387989   

                                                              
        A(not tp)        tp         A   I/(I+A) I/population  
2013-12       0.0  0.000000  0.000000  0.297521     0.016100  
2014-01       0.0  0.000000  0.000000  0.260870     0.013417  
2014-02       0.0  0.000000  0.000000  0.610092     0.059481  
2014-03       0.0  1.000000  1.000000  0.720241     0.097868  
2014-04       0.0  2.833333  2.833333  0.922400     0.451861  
2014-05       0.0  2.361111  2.361111  0.961763     0.956157  

[6 rows x 98 columns]

In [68]:
org_df.to_csv('SZFZ_monthly_ver2.csv')

In [57]:
df = pd.read_csv('SYZX_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])
df['DateConfirm'] = df['DateConfirm'].dt.date
df['DateConfirm'] = df['DateConfirm'].apply(lambda x: str(x.year) + '-' + str(x.month))
df.to_csv('SYZX_working_monthly_2.csv',index=None)

In [65]:
df.drop('DateConfirm',axis=1,inplace=True)
df.drop(94,inplace=True)

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateSymptom'] = df['DateSymptom'].dt.to_period('M')

df.loc[df['Class'].isnull(),'Class'] = 0

date_start = df.sort_values(by='DateSymptom').iloc[0]['DateSymptom']
date_end = df.sort_values(by='DateSymptom').iloc[-1]['DateSymptom']
date_index = pd.PeriodIndex(start=date_start,end=date_end,freq='m')

df.loc[:,'Class']=df.loc[:,'Class'].astype(float)

In [66]:
TYPES = ['I_new','I','A(not tp)','tp','A','I/(I+A)','I/population']
CLASSES = df['Class'].unique()
CLASSES.sort()
CLASSES = CLASSES[::-1]
CLASSES = np.append(CLASSES,[1,])
CLASSES = CLASSES.astype(float)

POPULATION = [1118/18,] * 19
POPULATION = np.append(POPULATION,[1118,])

type_class = [(t,c)for t in TYPES for c in CLASSES]

type_index = pd.MultiIndex.from_tuples(type_class)

org_df = pd.DataFrame(index=date_index,columns=type_index)

org_df[:] = 0

date_length = len(date_index)
for c in CLASSES[:-1]:
    class_df = df[df['Class']==c]
    for t,n in [('I',1),('A(not tp)',2),('tp',3)]:
        cur_df = class_df[class_df['Type']==n]['DateSymptom'].value_counts().sort_index()
        cur_dates = cur_df.index.values
        cur_vals = cur_df.values
        l = 0
        for i in range(date_length):
            if i>0:
                org_df.loc[date_index[i],(t,c)] = org_df.loc[date_index[i-1],(t,c)] * 5/6
            if l<len(cur_dates) and date_index[i]==cur_dates[l]:
                org_df.loc[date_index[i],(t,c)] += cur_vals[l]
                if n==1:
                    org_df.loc[date_index[i],('I_new',c)] = cur_vals[l]
                l += 1

date_length = len(date_index)
for c in CLASSES[:-1]:
    org_df.loc[:,('A',c)] = org_df.loc[:,('A(not tp)',c)] + org_df.loc[:,('tp',c)]

for t in ['I_new','I','A(not tp)','tp','A']:
    org_df.loc[:,(t,1)] = org_df.loc[:,t].iloc[:,:-1].sum(axis=1)

for c,p in zip(CLASSES,POPULATION):
    org_df.loc[:,('I/(I+A)',c)] = org_df.loc[:,('I',c)] / (org_df.loc[:,('I',c)] + org_df.loc[date_index[i],('A',c)])
    org_df.loc[:,('I/population',c)] = org_df.loc[:,('I',c)] / p

In [67]:
org_df=org_df.swaplevel(axis=1)
v = pd.Categorical(org_df.columns.get_level_values(0), 
                   categories=CLASSES, 
                   ordered=True)
v2 = pd.Categorical(org_df.columns.get_level_values(1), 
                    categories=TYPES,
                    ordered=True)
org_df.columns = pd.MultiIndex.from_arrays([v, v2]) 
org_df = org_df.sort_index(axis=1, level=[0, 1])
org_df.fillna(0,inplace=True)
org_df

52.0                                                51.0       \
        I_new    I A(not tp)   tp    A I/(I+A) I/population I_new    I   
2013-12     0  0.0       0.0  0.0  0.0     0.0       0.0000     0  0.0   
2014-01     0  0.0       0.0  0.0  0.0     0.0       0.0000     0  0.0   
2014-02     0  0.0       0.0  0.0  0.0     0.0       0.0000     0  0.0   
2014-03     0  0.0       0.0  0.0  0.0     0.0       0.0000     0  0.0   
2014-04     0  0.0       0.0  0.0  0.0     0.0       0.0000     0  0.0   
2014-05     1  1.0       0.0  0.0  0.0     1.0       0.0161     1  1.0   

                      ...      0.0                        1.0              \
        A(not tp)     ...         A I/(I+A) I/population I_new          I   
2013-12       0.0     ...       0.0     0.0     0.000000     1   1.000000   
2014-01       0.0     ...       0.0     0.0     0.000000     0   0.833333   
2014-02       0.0     ...       0.0     1.0     0.048301     3   3.694444   
2014-03       0.0     ...       0.0     1.0     0.072451     3   6.078704   
2014-04       0.0     ...       0.0     1.0     0.173077    23  28.065586   
2014-05       0.0     ...       0.0     1.0     0.144231    36  59.387989   

                                                              
        A(not tp)        tp         A   I/(I+A) I/population  
2013-12       0.0  0.000000  0.000000  0.297521     0.016100  
2014-01       0.0  0.000000  0.000000  0.260870     0.013417  
2014-02       0.0  0.000000  0.000000  0.610092     0.059481  
2014-03       0.0  1.000000  1.000000  0.720241     0.097868  
2014-04       0.0  2.833333  2.833333  0.922400     0.451861  
2014-05       0.0  2.361111  2.361111  0.961763     0.956157  

[6 rows x 98 columns]

In [68]:
org_df.to_csv('SZFZ_monthly_ver2.csv')

In [69]:
df = pd.read_csv('MBZX_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])
df['DateConfirm'] = df['DateConfirm'].dt.date
df['DateConfirm'] = df['DateConfirm'].apply(lambda x: str(x.year) + '-' + str(x.month))
df.to_csv('MBZX_working_monthly_2.csv',index=None)

In [71]:
df.drop(['DateSymptom','Class'],axis=1,inplace=True)

df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])
df['DateConfirm'] = df['DateConfirm'].dt.to_period('M')

date_start = df.sort_values(by='DateConfirm').iloc[0]['DateConfirm']
date_end = df.sort_values(by='DateConfirm').iloc[-1]['DateConfirm']
date_index = pd.PeriodIndex(start=date_start,end=date_end,freq='m')

TYPES = ['I+','I-','A','R']
org_df = pd.DataFrame(index=date_index,columns=TYPES)
org_df[:] = 0

date_length = len(date_index)
for t,n in [('I+',1),('I-',2),('A',3)]:
    cur_df = df[df['Type']==n]['DateConfirm'].value_counts().sort_index()
    cur_dates = cur_df.index.values
    cur_vals = cur_df.values
    l = 0
    for i in range(date_length):
        if i>0:
            org_df.loc[date_index[i],t] = org_df.loc[date_index[i-1],t] * 5/6
        if l<len(cur_dates) and date_index[i]==cur_dates[l]:
            org_df.loc[date_index[i],t] += cur_vals[l]
            l += 1

date_length = len(date_index)
for i in range(1,date_length):
    org_df.loc[date_index[i],'R'] = org_df.loc[date_index[i-1],'R'] + \
                                        org_df.loc[date_index[i-1],'I+'] /6 + \
                                        org_df.loc[date_index[i-1],'I-'] /6 + \
                                        org_df.loc[date_index[i-1],'A'] /6 

org_df.to_csv('MBZX_monthly.csv')
org_df

,I+,I-,A,R
2014-09,1.000000,3.000000,0.000000,0.000000
2014-10,0.833333,6.500000,4.000000,0.666667
2014-11,0.694444,6.416667,4.333333,2.555556


In [17]:
df = pd.read_csv('QJYS_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])
df['DateConfirm'] = df['DateConfirm'].dt.date
df['DateConfirm'] = df['DateConfirm'].apply(lambda x: str(x.year) + '-' + str(x.month))
df.to_csv('QJYS_working_monthly_2.csv',index=None)